In [1]:
import re
import os
import sys
import glob
import platform

if platform.system() == "Windows":
    sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/')
    sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/')
    sys.path.append('C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/model')
else: # Ubunutu
    sys.path.append('/home/user/Self_Driving_Car/')
    sys.path.append('/home/user/Self_Driving_Car/Imitation_Learning_RL/')
    sys.path.append('/home/user/Self_Driving_Car/Imitation_Learning_RL/model')


In [2]:
import gc
import torch
import multiprocessing as mp
from stable_baselines3 import SAC
from carla_gym_env import CarlaEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from custom_feature_extractor import UnifiedFeatureExtractor
from stable_baselines3.common.callbacks import CheckpointCallback, CallbackList, BaseCallback

['c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\python310.zip', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\DLLs', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15', '', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\win32', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\win32\\lib', 'c:\\Users\\WORKSTATION2\\Miniconda3\\envs\\carla_15\\lib\\site-packages\\Pythonwin', 'C:/Users/WORKSTATION2/Self_Driving_Car/', 'C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/', 'C:/Users/WORKSTATION2/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model', '/home/kaustubh/Documents/GitHub/Self_Driving_Car/Imitation_Learning_RL/model']


c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\WORKSTATION2\Miniconda3\envs\carla_15\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
CHECKPOINTS_DIR = "checkpoints"
TENSORBOARD_LOGS = "tensorboard_logs"
CHECKPOINT_PATH = os.path.join(CHECKPOINTS_DIR, "sac_carla_model")
TOTAL_TIMESTEPS = 5_000_000
SAVE_INTERVAL = 10000
DEVICE = "cuda" if torch.cuda.is_available() else 'cpu'

In [4]:
def make_env(host, port):
    def _init():
        env = CarlaEnv(host=host, port=port)
        env.reset()
        env = Monitor(env)
        return env
    return _init

In [5]:
pc_ip = "192.168.0.2"
# env_fns = [make_env(pc_ip, 2000 + i) for i in range(0, 19, 3)]
env_fns = [make_env(pc_ip, port) for port in [2000, 2003, 2006, 2009]]
# env_fns = [make_env(pc_ip, port) for port in [2000,2003, 2006, 2009]]


In [6]:
class TorchGCCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
    
    def _on_step(self) -> bool:
        if self.n_calls % 100 == 0:
            torch.cuda.empty_cache()
            gc.collect()
        return True

In [7]:
def find_latest_checkpoint(checkpoints_dir, prefix="sac_carla_model"):
    """Find the checkpoint with the highest number of steps."""
    checkpoint_files = glob.glob(os.path.join(checkpoints_dir, f"{prefix}_*steps.zip"))
    if not checkpoint_files:
        return None

    def extract_steps(path):
        match = re.search(rf"{prefix}_(\d+)_steps\.zip", os.path.basename(path))
        return int(match.group(1)) if match else -1

    checkpoint_files = sorted(checkpoint_files, key=extract_steps)
    return checkpoint_files[-1] 

In [ ]:
def main():

    
    os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
    os.makedirs(TENSORBOARD_LOGS, exist_ok=True)
    
    # try:
    #     env=SubprocVecEnv(env_fns)
    # except Exception as e:
    #     print(f"Failed to create multi-envs: {e}")
    #     env=DummyVecEnv([make_env("localhost", 2000)])

    env=SubprocVecEnv(env_fns)
    # env=DummyVecEnv([make_env("localhost", 2000)])

    # env=DummyVecEnv(env_fns)

    # print(env.observation_space)

    
    policy_kwargs = {
        'features_extractor_class' : UnifiedFeatureExtractor,

        'features_extractor_kwargs' : {
            "features_dim" : 768
        },
  
        # 'net_arch' : dict(pi=[768, 256, 256], qf=[768, 256, 256])
        # 'net_arch' : dict(pi=[512, 256], qf=[512, 256])
        # 'net_arch' : dict(pi=[256, 128], qf=[256, 128])
        'net_arch' : dict(pi=[128, 64], qf=[128, 64])
        
    }
    
    latest_checkpoint = find_latest_checkpoint(CHECKPOINTS_DIR)
    
    if latest_checkpoint:
        model = SAC.load(latest_checkpoint, env=env, device=DEVICE)
        print(f"Resuming training from checkpoint: {latest_checkpoint}")
    else:
        print("Starting new training.")
        
        model = SAC(
            "MultiInputPolicy",
            env,
            policy_kwargs=policy_kwargs,
            learning_rate=3e-4,
            buffer_size=1000,
            learning_starts=5000,
            batch_size=128,
            tau=0.005,
            gamma=0.99,
            train_freq=1,
            gradient_steps=1,
            tensorboard_log=TENSORBOARD_LOGS,
            verbose=1,
            device=DEVICE
        )
    
    checkpoint_callback = CheckpointCallback(save_freq=1000, save_path=CHECKPOINTS_DIR, name_prefix='sac_carla_model')
    callbacks = CallbackList([checkpoint_callback, TorchGCCallback()])
    timesteps_done = model.num_timesteps
    remaining_timesteps = TOTAL_TIMESTEPS - timesteps_done

    print(f"Already trained: {timesteps_done} steps. Remaining: {remaining_timesteps} steps.")

    model.learn(total_timesteps=remaining_timesteps, callback=callbacks, progress_bar=True, reset_num_timesteps=False)

    gc.collect()  
    torch.cuda.empty_cache()
    model.save("sac_carla_model")
    
    final_model_path = os.path.join(CHECKPOINTS_DIR, f"sac_final_{TOTAL_TIMESTEPS//1000}k")
    model.save(final_model_path)
    print(f"[DONE] Final model saved to: {final_model_path}")

In [11]:
if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)
    main()

Resuming training from checkpoint: checkpoints\sac_carla_model_28000_steps.zip
Already trained: 28000 steps. Remaining: 4972000 steps.
Logging to tensorboard_logs\SAC_0


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.6     |
|    ep_rew_mean     | -153     |
| time/              |          |
|    episodes        | 1276     |
|    fps             | 8        |
|    time_elapsed    | 9        |
|    total_timesteps | 28088    |
| train/             |          |
|    actor_loss      | 132      |
|    critic_loss     | 73.5     |
|    ent_coef        | 0.193    |
|    ent_coef_loss   | -5.71    |
|    learning_rate   | 0.0003   |
|    n_updates       | 5770     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.8     |
|    ep_rew_mean     | -148     |
| time/              |          |
|    episodes        | 1280     |
|    fps             | 8        |
|    time_elapsed    | 17       |
|    total_timesteps | 28152    |
| train/             |          |
|    actor_loss      | 129      |
|    critic_loss     | 52       |
|    ent_coef        | 0.192    |
|    ent_coef_loss   | -5.82    |
|    learning_rate   | 0.0003   |
|    n_updates       | 5786     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 13.3     |
|    ep_rew_mean     | -144     |
| time/              |          |
|    episodes        | 1284     |
|    fps             | 8        |
|    time_elapsed    | 27       |
|    total_timesteps | 28236    |
| train/             |          |
|    actor_loss      | 129      |
|    critic_loss     | 27.1     |
|    ent_coef        | 0.191    |
|    ent_coef_loss   | -5.39    |
|    learning_rate   | 0.0003   |
|    n_updates       | 5807     |
---------------------------------


...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.8     |
|    ep_rew_mean     | -81.9    |
| time/              |          |
|    episodes        | 1740     |
|    fps             | 8        |
|    time_elapsed    | 981      |
|    total_timesteps | 36192    |
| train/             |          |
|    actor_loss      | 99       |
|    critic_loss     | 4.6      |
|    ent_coef        | 0.112    |
|    ent_coef_loss   | -6.47    |
|    learning_rate   | 0.0003   |
|    n_updates       | 7796     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.7     |
|    ep_rew_mean     | -82.4    |
| time/              |          |
|    episodes        | 1744     |
|    fps             | 8        |
|    time_elapsed    | 991      |
|    total_timesteps | 36272    |
| train/             |          |
|    actor_loss      | 101      |
|    critic_loss     | 2.58     |
|    ent_coef        | 0.111    |
|    ent_coef_loss   | -5.73    |
|    learning_rate   | 0.0003   |
|    n_updates       | 7816     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.8     |
|    ep_rew_mean     | -82.3    |
| time/              |          |
|    episodes        | 1748     |
|    fps             | 8        |
|    time_elapsed    | 999      |
|    total_timesteps | 36340    |
| train/             |          |
|    actor_loss      | 101      |
|    critic_loss     | 1.49     |
|    ent_coef        | 0.111    |
|    ent_coef_loss   | -5.6     |
|    learning_rate   | 0.0003   |
|    n_updates       | 7833     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 17.6     |
|    ep_rew_mean     | -82.2    |
| time/              |          |
|    episodes        | 1752     |
|    fps             | 8        |
|    time_elapsed    | 1007     |
|    total_timesteps | 36404    |
| train/             |          |
|    actor_loss      | 100      |
|    critic_loss     | 3.7      |
|    ent_coef        | 0.11     |
|    ent_coef_loss   | -6.54    |
|    learning_rate   | 0.0003   |
|    n_updates       | 7849     |
---------------------------------


EOFError: 

In [ ]:
import carla
for port in [2000, 2003, 2006, 2009, 2012, 2015, 2018]:
    try:
        client = carla.Client('192.168.0.2', port)
        client.set_timeout(5.0)
        world = client.get_world()
        print(f"Connected to port {port}")
    except Exception as e:
        print(f"Failed to connect to port {port}: {e}")

In [ ]:
import carla

In [ ]:
client_ = carla.Client("localhost", 2000)
client_.set_timeout(10.0)
world_ = client_.get_world()
blueprint_library_ = world_.get_blueprint_library()

In [ ]:
for ve in blueprint_library_.filter("*vehicle*"):
    print(ve)